# Izdelava agentnega modela aktivnega odjemalca

Najprej samo en odjemalec ki je dovzeten na zunanje vplive aka community cordinator.

- Agent
    - 24 urni urnik porabe
        - Flexibile ali,
        - FIxed
    - Njegova utež, znak obnašanja -> Sprememba v fleksibilnost. (Ali je agent pripravljen premikati porabo)
    - Susceptibility oz. dovzetnost na spremembe [Jensen 2015] ali influencability [Douw 2016]

![](https://paper-attachments.dropbox.com/s_9E1224E4993919A665319145380AED8FB8A0E410E7EAEB27191C31F34972CCD9_1618215920264_image.png)




In [0]:
import pandas as pd
import random
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

# Uvoz podatkov

In [0]:
#load schedhule and load data
data = pd.read_excel(r'notebookData_flexiblestatic_load.xlsx')
data = data.set_index('time')
#store data in schedule dictionary key=hour, pair=type of load
schedule = {}
for index, row in data.iterrows():
    schedule[index] = row['load']
print(schedule)

{1: 'static', 2: 'static', 3: 'static', 4: 'static', 5: 'static', 6: 'static', 7: 'static', 8: 'static', 9: 'static', 10: 'static', 11: 'static', 12: 'static', 13: 'flexible', 14: 'flexible', 15: 'flexible', 16: 'flexible', 17: 'flexible', 18: 'flexible', 19: 'flexible', 20: 'flexible', 21: 'flexible', 22: 'flexible', 23: 'flexible', 24: 'flexible'}


# Ustvari agente

Funkcija za kreiranje agentov, ki vrne slovar agentov kjer je njihov id ključ, vrednosti pa predstavljajo urnik porabe in neka utež.

#todos:
- izdelava omrežja in povezav med agenti
- dodajanje primernih in različnih uteži
- dodajanje vrste consumerja
- dodajanje moči povezave med agenti

In [0]:
def create_agents(number_of_agents, schedule_data, flexibility, susceptibility):
    agent_list = {}
    while number_of_agents > 0: 
        agent = [schedule_data, flexibility, susceptibility]
        agent_list[number_of_agents] = agent
        number_of_agents -= 1
    return agent_list 

#ustvari slovar agentov z nastavljenimi vrednostmi
all_agents = create_agents(1,schedule,0.5,0.1)
all_agents

{1: [{1: 'static',
   2: 'static',
   3: 'static',
   4: 'static',
   5: 'static',
   6: 'static',
   7: 'static',
   8: 'static',
   9: 'static',
   10: 'static',
   11: 'static',
   12: 'static',
   13: 'flexible',
   14: 'flexible',
   15: 'flexible',
   16: 'flexible',
   17: 'flexible',
   18: 'flexible',
   19: 'flexible',
   20: 'flexible',
   21: 'flexible',
   22: 'flexible',
   23: 'flexible',
   24: 'flexible'},
  0.5,
  0.1]}

# Sprememba obnašanja

Funkcija za spremembo obnašanja agenta. Zaenkrat pregleda vse agente, kasneje je smiselno omejiti samo na enega za večjo fleksibilnst. 

Glavna sprememba se dogaja s parametrom fleksibilnosti, ki se izračuna po enačbi za feedback efekt [Jensen 2015], ki dobro pnazori zunanji vpliv na agenta. 

![](https://paper-attachments.dropbox.com/s_E6DE9BF2AA34A0A31A1F56F8C2F61B575F68978C991E079469C626C1C89ECF00_1594672402397_image.png)

*Po enačbi se feedback efekt izračuna tako, da se gleda trenuten behaviour (βt) in se ga primerja z prejšnim obnašanjem (βt-1), ki pa se mu prišteje razlika med INCENTIVIZED behaviour in prejšnjim obnašanjem (βt-1), vse skupaj pa se spreminja z stopnjo susceptibilitya za feedback effect (Δβ)
Incentivizied behaviour: Je obnašanje, ki ga spodbujamo, uporabnika skušamo motivirati proti tem obnašanju*

In [0]:
def behaviour(agent_list, hour):
    #preko vseh agentov
    for agent in agent_list:
        #nastavi fleksibilnost
        flexibility = agent_list[agent][1]
        #nastavi load 
        load = agent_list[agent][0][hour]
        #goal flexibility -> Koliko želimo da agent postane flksibilen
        goal_flexibility = 0.6
        print('Fleksibilnost na zacetku: ', flexibility)
        if load == 'static':
            # če je load static naredi nekaj z utežjo
            # priredba enačbe za izračun feedback efekta po Jensen, tako da posnamemo vpliv od zunaj
            flexibility = flexibility + (goal_flexibility - flexibility)* agent_list[agent][2]
            # update agent flexibility
            agent_list[agent][1] = flexibility
            if flexibility >= 0.6:
               load = 'flexible'
               agent_list[agent][0][hour] = load
            print('Fleksibilnost na koncu: ', flexibility)
            print('Trenutni load', load)
    return agent_list

behaviour(all_agents,1)
all_agents

Fleksibilnost na zacetku:  0.5
Fleksibilnost na koncu:  0.51
Trenutni load static


{1: [{1: 'static',
   2: 'static',
   3: 'static',
   4: 'static',
   5: 'static',
   6: 'static',
   7: 'static',
   8: 'static',
   9: 'static',
   10: 'static',
   11: 'static',
   12: 'static',
   13: 'flexible',
   14: 'flexible',
   15: 'flexible',
   16: 'flexible',
   17: 'flexible',
   18: 'flexible',
   19: 'flexible',
   20: 'flexible',
   21: 'flexible',
   22: 'flexible',
   23: 'flexible',
   24: 'flexible'},
  0.51,
  0.1]}

# Simulacija

Simulacija naj bi se izvajala več dni, da bomo kasneje posnemali vplive med agenti skozi vsakodnevne interakcije. Zato je ključno, da se uredi števec, ki se sprehaja skozi ure dneva za primerjavo z urnikom porabe agentov. 

In [0]:
def simulation(days, agent_list):
    # rabimo nek števec za ure ki predstavlja 1 dan
    hours = days * 24
    t = 0
    time = 1
    while days > 0:
        while t < hours:     
            print('day hour:', time)
            behaviour(agent_list, time)
            time +=1
            t +=1 

In [0]:
simulation(24, all_agents)

day hour: 1
Fleksibilnost na zacetku:  0.51
Fleksibilnost na koncu:  0.519
Trenutni load static
day hour: 2
Fleksibilnost na zacetku:  0.519
Fleksibilnost na koncu:  0.5271
Trenutni load static
day hour: 3
Fleksibilnost na zacetku:  0.5271
Fleksibilnost na koncu:  0.53439
Trenutni load static
day hour: 4
Fleksibilnost na zacetku:  0.53439
Fleksibilnost na koncu:  0.5409510000000001
Trenutni load static
day hour: 5
Fleksibilnost na zacetku:  0.5409510000000001
Fleksibilnost na koncu:  0.5468559000000001
Trenutni load static
day hour: 6
Fleksibilnost na zacetku:  0.5468559000000001
Fleksibilnost na koncu:  0.55217031
Trenutni load static
day hour: 7
Fleksibilnost na zacetku:  0.55217031
Fleksibilnost na koncu:  0.556953279
Trenutni load static
day hour: 8
Fleksibilnost na zacetku:  0.556953279
Fleksibilnost na koncu:  0.5612579511
Trenutni load static
day hour: 9
Fleksibilnost na zacetku:  0.5612579511
Fleksibilnost na koncu:  0.56513215599
Trenutni load static
day hour: 10
Fleksibilnost

KeyError: KeyError: 25

In [0]:
#test
dnevi = 4
while dnevi > 0: 
    ure = 24
    while ure > 0:
        ure += 1
        print("Ura: ", ure)
        print("Dan: ", dnevi)
    dnevi += 1


Ura:  23
Dan:  4
Ura:  22
Dan:  4
Ura:  21
Dan:  4
Ura:  20
Dan:  4
Ura:  19
Dan:  4
Ura:  18
Dan:  4
Ura:  17
Dan:  4
Ura:  16
Dan:  4
Ura:  15
Dan:  4
Ura:  14
Dan:  4
Ura:  13
Dan:  4
Ura:  12
Dan:  4
Ura:  11
Dan:  4
Ura:  10
Dan:  4
Ura:  9
Dan:  4
Ura:  8
Dan:  4
Ura:  7
Dan:  4
Ura:  6
Dan:  4
Ura:  5
Dan:  4
Ura:  4
Dan:  4
Ura:  3
Dan:  4
Ura:  2
Dan:  4
Ura:  1
Dan:  4
Ura:  0
Dan:  4
Ura:  23
Dan:  3
Ura:  22
Dan:  3
Ura:  21
Dan:  3
Ura:  20
Dan:  3
Ura:  19
Dan:  3
Ura:  18
Dan:  3
Ura:  17
Dan:  3
Ura:  16
Dan:  3
Ura:  15
Dan:  3
Ura:  14
Dan:  3
Ura:  13
Dan:  3
Ura:  12
Dan:  3
Ura:  11
Dan:  3
Ura:  10
Dan:  3
Ura:  9
Dan:  3
Ura:  8
Dan:  3
Ura:  7
Dan:  3
Ura:  6
Dan:  3
Ura:  5
Dan:  3
Ura:  4
Dan:  3
Ura:  3
Dan:  3
Ura:  2
Dan:  3
Ura:  1
Dan:  3
Ura:  0
Dan:  3
Ura:  23
Dan:  2
Ura:  22
Dan:  2
Ura:  21
Dan:  2
Ura:  20
Dan:  2
Ura:  19
Dan:  2
Ura:  18
Dan:  2
Ura:  17
Dan:  2
Ura:  16
Dan:  2
Ura:  15
Dan:  2
Ura:  14
Dan:  2
Ura:  13
Dan:  2
Ura:  12
Dan:  2
